# Wikidata API Example

This notebook demonstrates the usage of the Mediawiki API to get and save information on Wikidata.

* Create a new bot username and password: https://www.wikidata.org/wiki/Special:BotPasswords
* Create a file env.txt to save the values like that:
```
BOT_NAME=Metaodi@WikidataTrainingBot
BOT_PASSWORD=lkjdlkjjhoihnfjohshakklks
```

In [12]:
# install the dependencies
!pip install requests python-dotenv

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support


In [13]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os
import datetime

# load environment variables from .env files (to store sensitive information like passwords)
load_dotenv(dotenv_path='env.txt')

session = requests.Session()

## Request an item from the API

In [16]:
WIKIDATA_API_URL = 'https://www.wikidata.org/w/api.php'

# sandbox item
item = 'Q15397819'

In [17]:
res = session.get(WIKIDATA_API_URL, params={
    'action': 'wbgetentities',
    'ids': item,
    'format': 'json',
})
result = res.json()
result

{u'entities': {u'Q15397819': {u'aliases': {u'fr': [{u'language': u'fr',
      u'value': u'bac \xe0 sable 3'}],
    u'nan': [{u'language': u'nan', u'value': u'Test'}]},
   u'claims': {u'P31': [{u'id': u'Q15397819$db64e926-46bb-5001-a673-23dfbb1a0132',
      u'mainsnak': {u'datatype': u'wikibase-item',
       u'datavalue': {u'type': u'wikibase-entityid',
        u'value': {u'entity-type': u'item',
         u'id': u'Q14204246',
         u'numeric-id': 14204246}},
       u'hash': u'e5ef3cd1a671227be40e6978474d86d17df8cbb4',
       u'property': u'P31',
       u'snaktype': u'value'},
      u'rank': u'normal',
      u'type': u'statement'},
     {u'id': u'Q15397819$A79B0AF3-5ED6-41D2-ADBF-0C2239146B7F',
      u'mainsnak': {u'datatype': u'wikibase-item',
       u'datavalue': {u'type': u'wikibase-entityid',
        u'value': {u'entity-type': u'item', u'id': u'Q5', u'numeric-id': 5}},
       u'hash': u'ad7d38a03cdd40cdc373de0dc4e7b7fcbccb31d9',
       u'property': u'P31',
       u'snaktype': u'va

## Login to Wikidata

In [18]:
# login to wikidata
# Note: the login is saved in the requests session (i.e. in the cookies)
#       so make sure to use the same session for all subsequent calls
res = session.get(WIKIDATA_API_URL, params={
    'action': 'query',
    'meta': 'tokens',
    'type': 'login',
    'format': 'json',
})
tokens = res.json()['query']['tokens']

res = session.post(WIKIDATA_API_URL, data={
    'action': 'login',
    'lgname': os.getenv('BOT_NAME'),
    'lgpassword': os.getenv('BOT_PASSWORD'),
    'lgtoken': tokens['logintoken'],
    'format': 'json'
})
login = res.json()
login

{u'login': {u'reason': u'The supplied credentials could not be authenticated.',
  u'result': u'Failed'}}

## Update the description of an item (+CSRF)

In [31]:
# generate csrf token
res = session.get(WIKIDATA_API_URL, params={
    'action': 'query',
    'meta': 'tokens',
    'type': 'csrf',
    'format': 'json',
})
csrf = res.json()['query']['tokens']['csrftoken']
csrf

now = datetime.datetime.now()
# update description of sandbox item
res = session.post(WIKIDATA_API_URL, data={
    'action': 'wbsetdescription',
    'id': item,
    'token': csrf,
    'language': 'en',
    'value': 'My super wikidata bot, current time: %s' % now.isoformat(),
    'format': 'json',
})
res.json()

{'entity': {'descriptions': {'en': {'language': 'en',
    'value': 'My super wikidata bot, current time: 2019-11-02T20:14:42.285335'}},
  'id': 'Q15397819',
  'type': 'item',
  'lastrevid': 1043569760},
 'success': 1}

In [32]:
# request the item again
res = session.get(WIKIDATA_API_URL, params={
    'action': 'wbgetentities',
    'ids': item,
    'format': 'json',
})
result = res.json()
result['entities']['Q15397819']['descriptions']['en']

{'language': 'en',
 'value': 'My super wikidata bot, current time: 2019-11-02T20:14:42.285335'}

## Let's add a triple

To add a triple, we use the [`wbcreateclaim`](https://www.wikidata.org/w/api.php?action=help&modules=wbcreateclaim) action. Let's see what parameters are needed.

In [37]:
# generate csrf token
res = session.get(WIKIDATA_API_URL, params={
    'action': 'query',
    'meta': 'tokens',
    'type': 'csrf',
    'format': 'json',
})
csrf = res.json()['query']['tokens']['csrftoken']
csrf

# add triple to item
res = session.post(WIKIDATA_API_URL, data={
    'action': 'wbcreateclaim',
    'entity': item,
    'token': csrf,
    'snaktype': 'value',
    'property': 'P106',
    'value': {'entity-type': 'item', 'numeric-id': 212238},
    'format': 'json',
})
res.json()

{'error': {'code': 'invalid-snak',
  'info': 'Invalid snak data.',
  'messages': [{'name': 'wikibase-api-invalid-snak',
    'parameters': [],
    'html': {'*': 'Invalid snak data.'}}],
  '*': 'See https://www.wikidata.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'},
 'servedby': 'mw1315'}